
# Λ<sub>c</sub><sup>+</sup> Reconstruction using ROOT TMVA

As described in the [introduction](Introduction.md), it is possible to use the TMVA (Toolkit for Multivariate Data Analysis) with ROOT to perform multivariate analysis on high energy physics data.
Here is shown how to use 3 of the TMVA methods:

* Linear cuts 
* Neural Networks of type Multilayer Perceptron (MLP)
* Boosted Decision Tree (BDT)

The results of this analysis are presented in [result file](introduction.md).<br>
With the latest versions of ROOT it is possible to use all functions in python codes. This feature is called pyROOT, an example of its use is shown here.

<p>&nbsp;</p>

In [15]:
from ROOT import TMVA, TCut, TFile  #import ROOT necessary functions
from math import floor              #used to evaluate the minimum between two values

## Declaration of Factory

With the TFile class of ROOT the [TMVA_output_4_6.root](TMVA_output_4_6.root) output file is opened. <br>
Then the TMVA factory object is declared. The details can be found in chapter 3 of the [TMVA Users Guide](References/TMVAUsersGuide.pdf).
the options used in the declaration are described in the table:

Option          |Given Value     |Description
----------------|----------|---------------------------
V|False|Verbose flag
Color|True|Flag for colored output
DrawProgressBar|True|Draw progress bar to display training, testing and evaluation schedule 
Transformations||List of transformations to test. For example with "I;D;P;U;G" string identity, decorrelation, PCA, uniform and Gaussian transformations will be applied
AnalysisType|Classification|Set the analysis type

<br>


In [14]:
outputFile = TFile( "TMVA_output_4_6.root", 'RECREATE' )
TMVA.Tools.Instance()
factory = TMVA.Factory( "TMVAClassification", outputFile,
                       "!V:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification" )

## DataLoader Declaration, adding variables and setting the data set

The <b>AnalysisResults.root</b> file is defined as input (more details on its content are given in the [analysis results](README.md)). Once the Dataloader has been declared, the variables contained in the root trees of the input file are added. the AddVariable function used requires the following arguments:

* A string containing the name of the variable (the same as the root tree)
* A label for variable (if not present the variable name is used)
* data point type (second argument if the label for variable is missing)

It is also possible to define a combination of variables or an operation with variable using the syntax ":=" in the first argument. An example of this is represented by the <i>CtK0S</i> variable.<br><br>
Spectator Variables have also been added. These are part of the data set but are not used for multivariate analysis in the Training or Test phases. They can for example be used for correlation tests.<br><br>
More details on the variables used and their choice can be found in [analysis results](README.md).

In [5]:
#open input file
input= TFile.Open("AnalysisResults.root")

#declaration
dataloader = TMVA.DataLoader("dataset")

#Add Variable
dataloader.AddVariable( "massK0S", 'F' )
dataloader.AddVariable( "CosThetaStar", 'F' )
dataloader.AddVariable( "combinedProtonProb", 'F')
dataloader.AddVariable( "signd0", 'F')
dataloader.AddVariable( "tImpParV0", 'F')
dataloader.AddVariable( "tImpParBach", 'F' )
dataloader.AddVariable( "CtK0S := DecayLengthK0S*0.497/v0P", 'F')
dataloader.AddVariable( "cosPAK0S", 'F' )

#Add spectator variables
dataloader.AddSpectator( "nSigmaTOFpr", 'F' )
dataloader.AddSpectator( "nSigmaTPCpr", 'F' )
dataloader.AddSpectator( "nSigmaTPCpi", 'F' )
dataloader.AddSpectator( "nSigmaTPCka", 'F' )
dataloader.AddSpectator( "bachelorEta", 'F' )
dataloader.AddSpectator( "LcPt", 'F' )
dataloader.AddSpectator( "bachelorPt", 'F')
dataloader.AddSpectator( "massLc2K0Sp", 'F' )
dataloader.AddSpectator( "massLc2Lambdapi",  'F' )
dataloader.AddSpectator( "massLambda", 'F' )
dataloader.AddSpectator( "massLambdaBar", 'F' )
dataloader.AddSpectator( "V0positivePt", 'F' )
dataloader.AddSpectator( "V0negativePt", 'F' )
dataloader.AddSpectator( "dcaV0pos", 'F' )
dataloader.AddSpectator( "dcaV0neg", 'F' )
dataloader.AddSpectator( "v0Pt", 'F' )
dataloader.AddSpectator( "dcaV0", 'F' )
dataloader.AddSpectator( "V0positiveEta",'F' )

apriamo il dataset contenente signal and background trees

In [7]:
# signal and backg trees for training and testing 
signal      = input.Get( "treeList_2_25_2_25_Sgn")
background  = input.Get( "treeList_2_25_2_25_Bkg")

global event weights per tree (see below for setting event-wise weights) Diamo al Training e al Test lo stesso tree, questo verrà quindi diviso in due sottounità che serviranno separatamente per il training e il test.

In [8]:
signalWeight     = 1.0
backgroundWeight = 1.0

dataloader.AddSignalTree(signal, signalWeight)
dataloader.AddBackgroundTree(background, backgroundWeight)

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree treeList_2_25_2_25_Sgn of type Signal with 1762028 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree treeList_2_25_2_25_Bkg of type Background with 1473789 events


aggiungo dei Tagli sull'impulso trasverso per selezionare solo una parte degli eventi presenti nel tree

In [9]:
mycuts = TCut("LcPt<6 && LcPt>4")
mycutb = TCut("LcPt<6 && LcPt>4")

Applicato il taglio sul momento trasverso il numero di eventi da utilizzare nel training e nel test viene deciso come segue:

1) il training e il testing del segnale viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 100000

2) il training e il testing del Bakground viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 200000

Training e testing vengono ovviamente fatti con eventi diversi selezionati in maniera randomica all'interno del tree


In [10]:
nTrainingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTrainingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))
nTestingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTestingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))

Usiamo la funzione DataLoader.PrepareTrainingAndTestTree per applicare i cut agli eventi di input

In [11]:
dataloader.PrepareTrainingAndTestTree( mycuts, mycutb,"nTrain_Signal=nTrainingEventsSgn:nTest_Signal=nTestingEventsSgn:nTrain_Background=nTrainingEventsBkg:nTest_Background=nTestingEventsBkg:SplitMode=Random:NormMode=NumEvents:!V")


In [12]:
%%capture cap --no-stderr 
factory.BookMethod( dataloader, TMVA.Types.kCuts, "Cuts","!H:!V:FitMethod=MC:EffSel:VarProp=FSmart" )
#factory.BookMethod( dataloader, TMVA.Types.kBDT, "BDT","!H:!V:NTrees=850:MinNodeSize=2.5%:MaxDepth=3:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" )
#factory.BookMethod(dataloader, TMVA.Types.kMLP, "MLP","H:!V:NeuronType=tanh:VarTransform=N:NCycles=600:HiddenLayers=N+5:TestRate=5:!UseRegulator" )

# Train MVAs
#factory.TrainAllMethods()
# Test MVAs
#factory.TestAllMethods()
# Evaluate MVAs
#factory.EvaluateAllMethods() 
# Save the output.
outputFile.Close()

Factory                  : Booking method: Cuts
                         : 
                         : Use optimization method: "Monte Carlo"
                         : Use efficiency computation method: "Event Selection"
                         : Use "FSmart" cuts for variable: 'massK0S'
                         : Use "FSmart" cuts for variable: 'CosThetaStar'
                         : Use "FSmart" cuts for variable: 'combinedProtonProb'
                         : Use "FSmart" cuts for variable: 'signd0'
                         : Use "FSmart" cuts for variable: 'tImpParV0'
                         : Use "FSmart" cuts for variable: 'tImpParBach'
                         : Use "FSmart" cuts for variable: 'CtK0S'
                         : Use "FSmart" cuts for variable: 'cosPAK0S'


In [13]:
with open('std_output.txt', 'w') as f:
     f.write(cap.stdout)